In [4]:
import tensorflow as tf
import cv2
import numpy as np
import multiprocessing
from functools import partial
import itertools

from data import preproc as pp, evaluation
from data.generator import DataGenerator, Tokenizer
from data.reader import Dataset
from network.model import HTRModel

In [2]:
def next_batch(imgs, batch_size):
    """Return model predict parameters"""

    index = 0

    while True:
        if index >= len(imgs):
            index = 0
            break

        index_temp = index
        until = index_temp + batch_size
        index = until

        x_test = imgs[index_temp:until]
        x_test = pp.normalization(x_test)

        yield x_test

In [8]:
input_size = (1024, 128, 1)
max_text_length = 128
charset_base = '0123456789abcdefghijklmnopqrstuvwxyzåäöABCDEFGHIJKLMNOPQRSTUVWXYZÅÄÖ-., :'

#'0123456789abcdefghijklmnopqrstuvwxyzåäöABCDEFGHIJKLMNOPQRSTUVWXYZÅÄÖ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¬ '

tokenizer = Tokenizer(chars=charset_base, max_text_length=max_text_length)

"""dtgen = DataGenerator(source=source_path,
                     batch_size=16,
                     charset=charset_base,
                     max_text_length=max_text_length,
                     predict=True)"""

model = HTRModel(architecture='flor',
                input_size=input_size,
                vocab_size=tokenizer.vocab_size,
                beam_width=10,
                top_paths=10)

model.compile(learning_rate=0.001)
model.load_checkpoint(target='/home/erik/Riksarkivet/Projects/handwritten-text-recognition/output/1930_census_name_gotland_original_hdsr14_car_a_hdsr14_car_b_hdsr14_cvl_iam/flor/checkpoint_weights.hdf5')

In [10]:
img = pp.preprocess('/home/erik/Riksarkivet/Projects/handwritten-text-recognition/00000016-main-per_reg_nr_5-PrsNr_2.jpg', input_size=input_size, binarize=False)

x_test = pp.normalization([img])

with tf.device('/gpu:0'):
    predicts, probabilities = model.predict(x=x_test, ctc_decode=True)
    predicts = [[tokenizer.decode(x) for x in y] for y in predicts]

2021-09-13 08:45:16.313041: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-13 08:45:16.805047: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-09-13 08:45:16.807087: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


In [5]:
%%time

batch_path = '/home/erik/Riksarkivet/Projects/AIRAII/data/batches_raw/Struktur_exempel_v2'

read = Reader(batch_path)

batch = read.read_raw_batch()

persons = dict()

i = 0

imgs = []
imgs_pp = []
imgs_ppp = []

input_size = (1024, 128, 1)

for person_key in batch['00000016']:

    

    fields = dict()
    
    for field_key in batch['00000016'][person_key]:

        results_for_field = []

        #print(batch['00000016'][person_key][field_key])

        #print('efter')
            
        for img in batch['00000016'][person_key][field_key]:
    
            

            #img = pp.preprocess(img, input_size=input_size, binarize=True)
            #x_test = pp.normalization([img])
            imgs.append(img)

            #predicts, probabilities = model.predict(x_test, ctc_decode=True)
            #predicts = [[tokenizer.decode(x) for x in y] for y in predicts]

    i += 1

    print(i)



batch_size = 64

for b in range(0, len(imgs), batch_size):

    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        r = pool.map(partial(pp.preprocess, input_size=input_size, binarize=bin),
                        imgs[b:b + batch_size])
        imgs_pp.append(r)
        pool.close()
        pool.join()

flatten = [j for sub in imgs_pp for j in sub]
print(len(flatten))
steps = int(np.ceil(len(imgs) / 16))

#print(imgs[0])

x_test = pp.normalization(flatten)
with tf.device('/cpu:0'):
    predicts, probabilities = model.predict(x=x_test, steps=steps ,ctc_decode=True)
    predicts = [[tokenizer.decode(x) for x in y] for y in predicts]


1
2
3
4
5
6
7
8
9
10
11
12
115
2021-09-03 10:00:23.895118: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-03 10:00:23.915037: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3699850000 Hz
/home/erik/Riksarkivet/Projects/AIRAII/.venv/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
CPU times: user 23.3 s, sys: 3.61 s, total: 26.9 s
Wall time: 13.9 s


In [6]:
assert len(imgs) == len(predicts)



In [8]:
print(imgs[31])

/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/father/PrsNr/PrsNr_4.jpg


In [19]:
x = next_batch(imgs, 16)
for w in x:
    print(len(w))

16
16
16
16
16
5


In [2]:
import cv2
import easyocr

input_size = (1024, 128, 1)

img = pp.preprocess('/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000010/child2/PrsNr/PrsNr_4.jpg', input_size=input_size, binarize=True)

reader = easyocr.Reader(['en'], gpu=False)

#img = cv2.imread('/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000010/child2/PrsNr/PrsNr_4.jpg')

result_easyocr = reader.readtext(img)

print(result_easyocr)

/home/erik/Riksarkivet/Projects/AIRAII/.venv/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
[([[17, 67], [131, 67], [131, 128], [17, 128]], '39k', 0.10304374259132221)]


In [16]:
import easyocr

reader = easyocr.Reader(['en'], gpu=False)

result_easyocr = reader.readtext(img)   

print(result_easyocr)

/home/erik/Riksarkivet/Projects/AIRAII/.venv/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [7]:
for pred, path in zip(predicts, imgs):
    print(str(pred[0]) + '\t' + path)

69	/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/child5/FodelseOrt/FodelseOrt.jpg
69	/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/child5/PrsNr/PrsNr_1.jpg
69	/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/child5/PrsNr/PrsNr_2.jpg
69	/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/child5/PrsNr/PrsNr_3.jpg
69	/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/child5/PrsNr/PrsNr_4.jpg
90	/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/child5/AndraForalderNamn/AndraForalderNamn.jpg
69	/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/spouse2/EFNamn_FodelseOrt/EFNamn_FodelseOrt_rad_1.jpg
69	/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/spouse2/EFNamn_FodelseOrt/EFNamn_FodelseOrt_rad_2.jpg
69	/home/erik/Riksarkivet/Projects/AIRAII/data/Struktur_exempel_v2/00000016/spouse2/EFNamn_FodelseOrt/EF